## Analyze the time series data

In [1]:
import os
import pandas as pd

In [2]:
os.chdir('../')
os.getcwd()

'/home/vinus/projects/MRNN'

In [3]:
# convert .xlsx to .csv
# pd.read_excel('data/next_medication_doses_biweek_dataframe_v1.xlsx').to_csv('data/next_medication_doses_biweek_dataframe_v1.csv', index=False)

In [7]:
df = pd.read_csv('data/next_medication_doses_biweek_dataframe_v1.csv').drop(columns=['Unnamed: 0'])
df.head()

,Study ID,Date,ACEI,ACEI_DOSE,ACEI_PCT_TARGET,ARB,ARB_DOSE,ARB_PCT_TARGET,ARNI,ARNI_DOSE,...,SX_HYPOTENSION_RAASI,SX_BRADICARDIA_RAASI,SX_BRADICARDIA_BB,SX_HYPOTENSION_BB,SX_BRADICARDIA_MRA,SX_HYPOTENSION_MRA,SX_BRADICARDIA,SX_HYPOTENSION,Sex,Age
0,HF-0001,2020-01-29,NaN,NaN,0.0,NaN,NaN,0.0,Sacubitril/valsartan,48.0,...,False,False,False,False,False,False,False,False,1.0,61.0
1,HF-0001,2020-04-28,NaN,NaN,0.0,NaN,NaN,0.0,Sacubitril/valsartan,196.0,...,False,False,False,False,False,False,False,False,1.0,61.0
2,HF-0001,2020-09-16,NaN,NaN,0.0,NaN,NaN,0.0,Sacubitril/valsartan,98.0,...,False,False,False,False,False,False,False,False,1.0,61.0
3,HF-0002,2020-01-29,NaN,NaN,0.0,NaN,NaN,0.0,Sacubitril/valsartan,48.0,...,False,False,False,False,False,False,False,False,1.0,63.0
4,HF-0002,2020-02-19,NaN,NaN,0.0,NaN,NaN,0.0,Sacubitril/valsartan,73.0,...,False,False,False,False,False,False,False,False,1.0,63.0


In [13]:
df.columns.size

90